BUILT-IN TOOLS


In [27]:
# DuckDuckGo search
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('ipl news')
print(results)
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

3 days ago · ... IPL matches, SRH players, Orange Army fan club, SRH highlights, Cricket news, SRH scores, Hyderabad cricket fans, IPL fan page. more. January 27. 2 hours ago · In cricket-crazy India, IPL redefined the sport over two decades · Global PE firms get more interested as IPL valuations rise · Cricket as an asset class has come ... 18 Dec 2025 · The official IPL website with live scores, match updates, team rankings, all the latest news, and videos . Follow your favorite teams and players! 2 days ago · Bengaluru's Chinnaswamy Stadium set to host IPL 2026 matches under strict safety mandates following the tragic 2025 stampede. Can accountability catch up? 14 Nov 2025 · News #TATAIPL 2026 - Player Trade updates Ahead of the IPL 2026 Retention Deadline, the franchises have agreed to the following player trades.
duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'descr

In [10]:
# Shell Tool : used for terminal responses
from langchain_community.tools import ShellTool
shell_tool=ShellTool()
results=shell_tool.invoke('whoami')
print(results)

Executing command:
 whoami
abhishekduggal\hp



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("BRAVE_SEARCH_API_KEY")
from langchain_community.tools import BraveSearch
search_tool = BraveSearch(brave_api_key=api_key)
results = search_tool.invoke('GATE 2026 news')
results

CUSTOM TOOLS 


METHOD 1 using @tool decorator (used in most of the cases)

In [18]:
#A docstring in Python is a special string literal that appears at the beginning of a module, class, or function, used to document what the code does. It helps programmers understand the purpose and usage of the code without reading the whole implementation.
#written inside """ """
# these are highly recommended iski help se future mei llm ko smjh mei ayega vi yeh tool krta kya hai 
#STEP 1 create a function
def multiply(a,b):
    """Multiply two numbers"""
    return a*b


In [19]:
#STEP 2 add type hints
def multiply(a:int, b:int) ->int:
    """Multiply two numbers"""
    return a*b

In [21]:
from langchain.tools import tool
#STEP 3 - add tool decorator
@tool # llm iske func ke sth communicate kr skta hai 
def multiply(a: int, b: int) ->int:
    """Multiply two numbers"""
    return a*b

In [22]:
result = multiply.invoke({"a":3 , "b":5})

In [ ]:
print("Result is : ",result)
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.model_json_schema()) # jab tool ko connect krege llm se toh yeh llm ko kya dikhayi deta hai voh btata hai

Result is :  15
multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


METHOD 2 - Using StructuredTool

In [33]:
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field

In [40]:
class MultiplyInputs(BaseModel):
    #... In Pydantic, that syntax is not supported. Pydantic does not recognize required=True inside Field()
    #... → Marks the field as required.
    a: int = Field(..., description="The first number you want to multiply")
    b: int = Field(..., description="The second number you want to multiply")

In [41]:
def multiply_func(a: int, b: int) -> int:
    return a*b

In [44]:
multiply_tools = StructuredTool.from_function (
    func = multiply_func,
    name = "MUL",
    description = "MUL BOTH THE NUMBERS",
    args_schema = MultiplyInputs
)

In [47]:
print("Result is : ",result)
print(multiply_tools.name)
print(multiply_tools.description)
print(multiply_tools.args)
print(multiply_tools.args_schema.model_json_schema()) # jab tool ko connect krege llm se toh yeh llm ko kya dikhayi deta hai voh btata hai

Result is :  15
MUL
MUL BOTH THE NUMBERS
{'a': {'description': 'The first number you want to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number you want to multiply', 'title': 'B', 'type': 'integer'}}
{'properties': {'a': {'description': 'The first number you want to multiply', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number you want to multiply', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInputs', 'type': 'object'}


METHOD 3 : BASE TOOL

In [49]:
from langchain.tools import BaseTool
from typing import Type

In [51]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(..., description="The first number to add")
    b: int = Field(..., description="The second number to add")

In [52]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [56]:
multiply_tool = MultiplyTool()
result = multiply_tool.invoke({'a':3, 'b':5})
result

15